<a href="https://colab.research.google.com/github/DSGP-Group-1-EAPS/SL-Apparel-Dataset/blob/main/CatboostClassificationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
!pip install scikit-learn==1.4.1.post1
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta
import numpy as np
!pip install catboost==1.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.4 MB/s eta 0:00:00


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Mounting the Google Drive
drive.mount('/content/drive')

training_df = pd.read_excel('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/training_dataset.xlsx')
retraining_df = pd.read_excel('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/retraining_dataset.xlsx')
sept_testing_df = pd.read_excel('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/sept_train_test.xlsx')

Mounted at /content/drive


In [ ]:
training_df.shape

(83236, 24)

In [ ]:
training_df.head()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
0,0,2021-12-01,Shift A,AA369,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2015-09-11,369,6,3,12,2021,MAT,1,10,1,0,1,127,0,A
1,1,2021-12-01,Shift A,AA362,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2018-11-14,362,3,3,12,2021,MAT,1,10,1,0,1,127,24,B
2,2,2021-12-01,Shift A,AA359,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2019-10-09,359,2,3,12,2021,MAT,1,10,1,0,1,127,16,B
3,3,2021-12-01,Shift A,AA541,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2020-10-02,541,1,3,12,2021,MAT,1,10,1,0,1,127,2,A
4,4,2021-12-01,Shift A,AA398,Team - MAT 3A - BD,Informed,Notified,1.0,Maternity,2008-02-29,398,13,3,12,2021,MAT,1,10,1,0,1,127,1,A


In [ ]:
training_df.tail()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
83231,83231,2023-07-31,Shift B,AA2877,Sewing Team - 147B - I - BD,Informed,Notified,1.0,Health Related,2021-06-29,2877,2,1,7,2023,Sewing Team,2,8,1,0,2,864,0,A
83232,83232,2023-07-31,Shift B,AA3478,Sewing Team - 121B - I - BD,Uninformed,Not Notified,1.0,No Message,2021-07-13,3478,2,1,7,2023,Sewing Team,2,13,0,1,2,864,0,A
83233,83233,2023-07-31,Shift B,AA4298,Sewing Team - 123B - I - BD,Uninformed,Not Notified,1.0,No Message,2022-03-01,4298,1,1,7,2023,Sewing Team,2,13,0,1,2,864,0,A
83234,83234,2023-07-31,Shift B,AA2830,Team - MAT 2B - BD,Informed,Notified,1.0,Maternity,2021-10-07,2830,1,1,7,2023,MAT,1,10,1,0,2,161,12,B
83235,83235,2023-07-31,Shift B,AA4275,Sewing Team - 155B - I - BD,Informed,Notified,1.0,Personal Reason,2022-10-07,4275,0,1,7,2023,Sewing Team,2,14,1,0,2,864,17,B


In [ ]:
retraining_df.shape

(85090, 24)

In [ ]:
retraining_df.head()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
0,0,2021-12-01,Shift A,AA369,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2015-09-11,369,6,3,12,2021,MAT,1,10,1,0,1,127,0,A
1,1,2021-12-01,Shift A,AA362,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2018-11-14,362,3,3,12,2021,MAT,1,10,1,0,1,127,24,B
2,2,2021-12-01,Shift A,AA359,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2019-10-09,359,2,3,12,2021,MAT,1,10,1,0,1,127,16,B
3,3,2021-12-01,Shift A,AA541,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2020-10-02,541,1,3,12,2021,MAT,1,10,1,0,1,127,2,A
4,4,2021-12-01,Shift A,AA398,Team - MAT 3A - BD,Informed,Notified,1.0,Maternity,2008-02-29,398,13,3,12,2021,MAT,1,10,1,0,1,127,1,A


In [ ]:
retraining_df.tail()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
85085,85085,2023-08-31,Shift A,AA3577,Sewing Team - 152A - I - BD,NaN,Not Notified,1.0,Medical Leave,2021-10-26,3577,1,4,8,2023,Sewing Team,2,11,0,2,1,852,8,B
85086,85086,2023-08-31,Shift B,AA3595,Sewing Team - 104B - I - BD,NaN,Not Notified,1.0,Personal Reason,2021-11-20,3595,1,4,8,2023,Sewing Team,2,14,0,2,2,852,1,A
85087,85087,2023-08-31,Shift A,AA3426,Sewing Team - 119A - I - BD,NaN,Not Notified,1.0,Medical Leave,2021-03-16,3426,2,4,8,2023,Sewing Team,2,11,0,2,1,852,2,A
85088,85088,2023-08-31,Shift A,AA4126,Sewing Team - 140A - I - BD,NaN,Not Notified,1.0,Health Related,2022-12-15,4126,0,4,8,2023,Sewing Team,2,8,0,2,1,852,1,A
85089,85089,2023-08-31,Shift A,AA2155,Sewing Team - 146A - II - BD,NaN,Not Notified,1.0,Personal Reason,2013-11-18,2155,9,4,8,2023,Sewing Team,2,14,0,2,1,852,6,B


In [ ]:
sept_testing_df.shape

(2317, 13)

In [ ]:
sept_testing_df.head()

,Unnamed: 0,Encoded Code,Encoded Department,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,TargetCategory
0,98398,365,2,24,5,9,2023,15,1,0,1,1003,A
1,98399,2278,1,14,5,9,2023,10,1,0,1,133,B
2,98400,393,1,13,5,9,2023,10,1,0,1,133,B
3,98402,4102,1,1,5,9,2023,10,1,0,1,133,B
4,98403,1442,2,9,5,9,2023,10,1,0,1,1003,B


In [ ]:
sept_testing_df.tail()

,Unnamed: 0,Encoded Code,Encoded Department,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,TargetCategory
2312,103784,4250,2,0,4,9,2023,14,1,0,1,1003,B
2313,103785,4278,2,1,4,9,2023,14,1,0,1,1003,A
2314,103786,3950,2,1,5,9,2023,14,1,0,1,1003,A
2315,103787,1143,2,8,5,9,2023,14,1,0,1,1003,A
2316,103788,3922,2,1,5,9,2023,14,1,0,1,1003,B


In [ ]:
features  = ['Encoded Code', 'Encoded Department', 'YearsWorked', 'DayOfWeek',
             'LeaveMonth', 'LeaveYear', 'Encoded Reason','Encoded Status',
             'Encoded Absenteeism Type', 'Encoded Shift', 'MonthlyDeptTotal']

In [ ]:
X_train = training_df[features]
Y_train = training_df['TargetCategory']

In [ ]:
X_retrain = retraining_df[features]
Y_retrain = retraining_df['TargetCategory']

In [ ]:
X_retrain.shape

(85090, 11)

In [ ]:
X_test = sept_testing_df[features]
Y_test = sept_testing_df['TargetCategory']

In [ ]:
X_test.head()

,Encoded Code,Encoded Department,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal
0,365,2,24,5,9,2023,15,1,0,1,1003
1,2278,1,14,5,9,2023,10,1,0,1,133
2,393,1,13,5,9,2023,10,1,0,1,133
3,4102,1,1,5,9,2023,10,1,0,1,133
4,1442,2,9,5,9,2023,10,1,0,1,1003


#Model Training

In [ ]:
from catboost import CatBoostClassifier

# Define and train the CatBoost model with proper parameters
model_catboost = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, loss_function='Logloss')
model_catboost.fit(X_train, Y_train)

In [ ]:
# Get predictions on the test set
predictions_catboost_training = model_catboost.predict(X_retrain)

# Calculate accuracy on the test set
accuracy_catboost_train = accuracy_score(Y_retrain, predictions_catboost_training)
print("CatBoost Accuracy:", accuracy_catboost_train)

check accuracy

In [ ]:
from sklearn.metrics import accuracy_score

# Get predictions on the test set
predictions_catboost = model_catboost.predict(X_test)

# Calculate accuracy on the test set
accuracy_catboost = accuracy_score(Y_test, predictions_catboost)
print("CatBoost Accuracy:", accuracy_catboost)

saving the model

In [ ]:
import pickle
# Save the model
with open('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Models/Catboost_model_original.pkl', 'wb') as file:
    pickle.dump(model_catboost, file)